In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/scraping-volleyball-teams/__results__.html
/kaggle/input/scraping-volleyball-teams/__notebook__.ipynb
/kaggle/input/scraping-volleyball-teams/scraped_team_info.csv
/kaggle/input/scraping-volleyball-teams/__output__.json
/kaggle/input/scraping-volleyball-teams/custom.css


In [2]:
# base url
base_url = "http://www.bvbinfo.com/player.asp?ID="

In [3]:
# file generated from another scraping script/notebook
team_df = pd.read_csv("/kaggle/input/scraping-volleyball-teams/scraped_team_info.csv")
team_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Finish           574 non-null    float64
 1   Player           611 non-null    object 
 2   Partner          611 non-null    object 
 3   Player_ID        611 non-null    int64  
 4   Partner_ID       611 non-null    int64  
 5   Country          611 non-null    object 
 6   TourniStartDate  611 non-null    object 
 7   TourniEndDate    611 non-null    object 
 8   TournLocation    611 non-null    object 
 9   Event            611 non-null    object 
 10  TOURNID          611 non-null    int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 52.6+ KB


In [4]:
scraped_df = pd.DataFrame(columns=["PLAYER_ID", "ATTRIBUTE", "VALUE"])
scraped_df

,PLAYER_ID,ATTRIBUTE,VALUE


In [5]:
%%time

columns = ["PLAYER_ID", "ATTRIBUTE", "VALUE"]

# Create a list to store the data
data = []
data_values = []
player_id_list = team_df['Player_ID'].unique()
print(len(player_id_list))

for player_id in player_id_list:
# for player_id in range(1,20):
    url = base_url + str(player_id)

    # Make a request to the URL
    try:
        response = requests.get(url, timeout=10)
    except requests.exceptions.Timeout as e:
        # If the request times out, skip to the next player ID
        print(f"Time out for player ID {player_id}: {e}")
        continue

    except requests.exceptions.RequestException as e:
        # If there is another error, skip to the next player ID
        print(f"Error for player ID {player_id}: {e}")
        continue

    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, "html.parser")

    player_table_element = soup.find('table', {'class': 'clsPlayerTable'})
    
    # Get player name and country
    name_element = player_table_element.find(class_="clsPlayerName")
    country_element = player_table_element.find(class_="clsPlayerCountry")
    name = name_element.text.strip()
    country = country_element.text.strip()
    
    data.append([player_id, "Name", name])
    data.append([player_id, "Country", country])
    
    # tables
    tables =player_table_element.find_all('table')
    for table_element in tables:
        header_element = table_element.find('td',{'class':"clsPlayerHeader"})
        if header_element:
            header_text = header_element.text.strip()
            # print(header_text)
            if 'Vital Statistics' == header_text:
                info_table = table_element.find('table').find('table')
                if info_table:
                    info_rows = info_table.find_all('tr')
                    for rows in info_rows:
                        label = rows.find(class_="clsPlayerDataLabel")
                        if label:
                            label = label.text.strip()
                        value = rows.find(class_="clsPlayerData")
                        if value:
                            value = value.text.strip()
                        data.append([player_id, label, value])              
            if 'Career Summary' == header_text:               
                categories = table_element.find_all('td',{'class':'clsPlayerCategoryHeader'})
                totals = table_element.find_all('tr',{'class':'clsPlayerDataTotal'})
                for i in range(len(categories)):
                    data.append([player_id, "Career-" + categories[i].text.strip(),totals[i].find_all('td')[2].text.split('(')[0]])
                overall_total = totals[len(totals)-1].find_all('td')[2].text.split('(')[0]
                data.append([player_id, "Career-total",overall_total])
# Display the scraped DataFrame
scraped_df = pd.DataFrame(data, columns=columns)

142
CPU times: user 39.6 s, sys: 118 ms, total: 39.7 s
Wall time: 1min 3s


In [6]:
%time
scraped_df.to_csv("scraped_player_data.csv", index = False)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs
